<a href="https://colab.research.google.com/github/AhzamAhmed6/CIFAR-using-CNN/blob/main/28PyTorch_CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import torchvision
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms

In [16]:
train_dataset = torchvision.datasets.CIFAR10(
    root='.',
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

test_dataset = torchvision.datasets.CIFAR10(
    root='.',
    train=False,
    download=False,
    transform=transforms.ToTensor()
)

batch_size=128
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=batch_size
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    shuffle=True,
    batch_size=batch_size
)

Files already downloaded and verified


In [17]:
train_dataset.data.shape

(50000, 32, 32, 3)

In [19]:
len(train_dataset.targets)

50000

In [21]:
[21,23,323,1213].np.shape

AttributeError: ignored